In [21]:

n_gram = None
is_laplace_smoothing = True
vocab = [] 
n_gram_counts = {}
n_minus_1_gram_counts = None
threshold = 1
n = 2

In [22]:
def make_ngrams(tokens: list, n: int) -> list:
    
    n_grams = []
    for i in range(len(tokens) - n + 1):
        temp = tuple(tokens[i: i + n])
        n_grams.append(temp)
    return n_grams


In [23]:
print(make_ngrams(["This","is","a","list"] , n))

[('This', 'is'), ('is', 'a'), ('a', 'list')]


In [24]:
def train(training_file_path):
    
    with open(training_file_path, 'r') as fh:
        content = fh.read().split()
    
    #print(len(content))
    #print(content)
    freq_word = {}
    for word in content:
        #if word != '<s>' or word != '</s>':
            if word not in freq_word.keys():
                freq_word[word] = 1
            else:
                freq_word[word] += 1
    #print(freq_word)
    
    
    
    for i in range(len(content)):
        if freq_word[content[i]] == 1:
            content[i] = '<UNK>'

    #print(content)

    
    vocab = set(content)
    
    n_grams_bi = make_ngrams(content , n )
    n_gram_counts = {}
    for ngrm in n_grams_bi : 
        if ngrm not in n_gram_counts.keys():
            n_gram_counts[ngrm] = 1
        else:
            n_gram_counts[ngrm] += 1
    
    n_minus_1_gram = make_ngrams(content , n-1 )
    n_minus_1_gram_counts = {}
    for ngrm in n_minus_1_gram : 
        if ngrm not in n_minus_1_gram_counts.keys():
            n_minus_1_gram_counts[ngrm] = 1
        else:
            n_minus_1_gram_counts[ngrm] += 1
    #print(n_minus_1_gram_counts)
            
    return n_gram_counts, vocab, n_minus_1_gram_counts

In [25]:
n_gram_counts, vocab, n_minus_1_gram_counts = train("/Users/rohitnair/Downloads/data-1/train_data/berp-training_bi.txt")
print(n_gram_counts)
print(vocab)
print(n_minus_1_gram_counts)


{('<s>', "let's"): 196, ("let's", 'start'): 170, ('start', 'over'): 403, ('over', '</s>'): 367, ('</s>', '<s>'): 6755, ('<s>', 'my'): 6, ('my', 'mother'): 2, ('mother', 'is'): 1, ('is', 'coming'): 1, ('coming', 'to'): 1, ('to', 'visit'): 2, ('visit', 'and'): 1, ('and', "i'd"): 5, ("i'd", 'like'): 495, ('like', 'to'): 953, ('to', 'take'): 12, ('take', '<UNK>'): 3, ('<UNK>', 'to'): 14, ('to', 'dinner'): 12, ('dinner', '</s>'): 224, ('<s>', 'new'): 1, ('new', 'query'): 1, ('query', '</s>'): 2, ('<s>', 'now'): 7, ('now', "i'm"): 8, ("i'm", 'interested'): 35, ('interested', 'in'): 39, ('in', 'some'): 5, ('some', 'middle'): 5, ('middle', 'eastern'): 14, ('eastern', 'food'): 10, ('food', '</s>'): 670, ('<s>', 'oh'): 29, ('oh', 'i'): 11, ('i', 'have'): 91, ('have', 'to'): 17, ('to', 'breakfast'): 1, ('breakfast', '</s>'): 89, ('i', 'increase'): 1, ('increase', 'the'): 2, ('the', 'walking'): 3, ('walking', 'distance'): 47, ('distance', 'i'): 3, ('i', 'can'): 69, ('can', 'go'): 14, ('go', 'fifte

In [26]:
def score(sentence):
    
    probablity = 0.0
    
    sent = sentence.split()
    #print(sent)
    for i in range(len(sent)):
        if sent[i] not in vocab:
            sent[i] = "<UNK>"

    #print(sent)
    #print(len(sent))
    
    #print(len(sent))
    if n >= 1:
        ngram_bi = make_ngrams(sent , n )
        #print(ngram_bi)
        #print(len(ngram_bi))
        sum_log = 0
        for i in range(len(ngram_bi)):
            if ngram_bi[i] not in n_gram_counts.keys():
                neum = 0
            else: 
                neum = n_gram_counts[ngram_bi[i]]
            if ngram_bi[i] not in n_gram_counts.keys():
                denom = 0
            else:
                denom = n_minus_1_gram_counts[tuple(ngram_bi[i][0:n-1])]
            #print(ngram_bi[i][0])
            #print(denom)
            V = len(vocab)
            if is_laplace_smoothing:
                probablity = (neum  + 1) / (denom + V)
            #print(probablity)
            sum_log += np.log10(probablity)
        probablity = 10**sum_log
    '''
    else:
        sum_log = 0
        freq_words = {}
        for word in sent:
        #if word != '<s>' or word != '</s>':
            if word not in freq_words.keys():
                freq_words[word] = 1
            else:
                freq_words[word] += 1
        for word in sent:
            probablity = freq_words[word] / len(sent)
            sum_log += np.log10(probablity)
        probablity = 10**sum_log
    '''
    return probablity

In [27]:
probablities = score("<s> how far is the international house cafe from the international computer science institute </s>")
print(probablities)



3.2104628576863764e-31


In [28]:
with open("/Users/rohitnair/Downloads/data-1/test_data/hw2-test_bi.txt", 'r') as fh:
    test_content = fh.read().split("\n")
num_sentences_1 = len(test_content)
ten_sentences_1 = test_content[:10]
print("# of test sentences: ", num_sentences_1)
probablities = []

# of test sentences:  100


In [29]:
for sentence in test_content:
    probablities.append(score(sentence))
probablities = np.array(probablities)
mean = np.mean(probablities)
std_dev = np.std(probablities)

print("Output for bigram model is - ")
print(mean)
print(std_dev)

Output for bigram model is - 
4.9682323990330295e-05
0.00028528822664480863


In [30]:
def get_choices(ngram):
    choices = {}
    for key , val in n_gram_counts.items():
        if key[0:n-1] == ngram[0:n-1]:
            choices[key] = val
    return choices

In [31]:
#delete
import random
sentence = ""
prev_words = ["<s>"] * (n-1)
print(prev_words)
for o in prev_words:
    sentence += " " + o 
for j in range(0,10):
    ngram = tuple(prev_words)
    choices = get_choices(ngram)
    total_count = sum(choices.values())
    #probabilities = {word: count / total_count for word, count in choices.items()}
    probabilities = {word : score(' '.join(word)) for word , count in choices.items()}
    #print(probabilities)
    word = random.choices(list(probabilities.keys()), list(probabilities.values()))[0]
    #print(word)
    if word[-1] == "<s>":
        continue
    else:    
        sentence += " " + word[-1]
        prev_words = word[1:]
        print(sentence)

['<s>']
 <s> i
 <s> i would
 <s> i would like
 <s> i would like to
 <s> i would like to eat
 <s> i would like to eat on
 <s> i would like to eat on the
 <s> i would like to eat on the four
 <s> i would like to eat on the four dollars
 <s> i would like to eat on the four dollars </s>


In [32]:
def generate_sentence():
    
    import random
    sentence = ""
    #n = 2
    if n >1:
        
        prev_words = ["<s>"] * (n-1)
        for o in prev_words:
            sentence += " " + o 
        while prev_words[-1] != "</s>":
            
            ngram = tuple(prev_words)
            choices = get_choices(ngram)
            #total_count = sum(choices.values())
            #probabilities = {word: count / total_count for word, count in choices.items()}
            probabilities = {word : score(' '.join(word)) for word , count in choices.items()}
            #print(probabilities)
            word = random.choices(list(probabilities.keys()), list(probabilities.values()))[0]
            #print(word)
            if word[-1] == "<s>":
                continue
            else:    
                sentence += " " + word[-1]
                prev_words = word[1:]
        if n > 2:
            sentence = "<s>"*(n-2) + sentence
            sentence += "</s>"*(n-2)
        
        #print(sentence)
        
                
    else:
        prev_words = ["<s>"]
        while prev_words[0] != "</s>":
            
            #n_minus1_grm = ()
            choices = { }
            probabilities = {word : score(' '.join(word)) for word in vocab}
            word = random.choices(list(probabilities.keys()), list(probabilities.values()))[0]
            #for i in range(len(vocab)):
                #if key[0:n-1] == ngram[0:n-1]:
                    #choices[key] = val

            if word[-1] == "<s>":
                continue
            else:    
                sentence += " " + word[-1]
                prev_words = word[-1]
    
    sentence.strip()
    return sentence

In [33]:
def generate(n):
    
    sentences = [] 
    for t in range(0,n):
        senten = generate_sentence()
        sentences.append(senten)
    return sentences

In [34]:
sentences = generate(50)
print("Sentences:")
for sentence in sentences:
    print(sentence)

Sentences:
 <s> what restaurants </s>
 <s> start over </s>
 <s> i would like to drive so within five blocks </s>
 <s> what about petrouchka picante taqueria cancun </s>
 <s> where is there any day </s>
 <s> tell me all italian restaurant near campus </s>
 <s> not too expensive is a moderately expensive pizza pizza blue nile </s>
 <s> give me kosher food on sunday for ten <UNK> dinner </s>
 <s> you tell me about restaurants you gave me about five to know a sunday </s>
 <s> it should be between say short i want thai restaurants you have a mile </s>
 <s> this will probably be one </s>
 <s> i want cheap japanese </s>
 <s> anywhere in berkeley </s>
 <s> ultra cheap </s>
 <s> i would like to eat a typical american food </s>
 <s> tell me about manuel's mario's la tour </s>
 <s> on saturday night </s>
 <s> i can you tell me more information on wednesday evening </s>
 <s> tell me about continental garden </s>
 <s> i'd like to eat italian restaurant that's fairly cheap side </s>
 <s> i want to g

In [35]:

def perplexity(test_sequence):
    

    test_sequence = [token if token in vocab else UNK for token in test_sequence.split()]


    test_sequence_N = [token for token in test_sequence if token != '<s>'  ]
    #print(test_sequence_N)
    N = len(test_sequence_N)
    
    
    probablities = score(' '.join(test_sequence))
    #print(probablities)

    probablities = np.log10(probablities)
    perplexity = 10**(-probablities / N)

    perplexity = perplexity
    return perplexity

In [36]:
print(perplexity(" ".join(sentences[0:10])))

64.26583856182039


#Notes -

We notice that as we increase the value of n , we get more meaningful sentences. But as n increases towards 7 , the result we get for sentence we generation is more restricted. This may be because we have increased the context length to 7 and the train model doesnt have too many long sentences with 7 or more words. Thus the train sample size is less.

Below is the results for 1-7 gram models

Output For Unigram model - 

mean = 2.5597522032540794e-06

std_dev = 1.4458481319522847e-05



Output For trigram model - 

mean = 0.019614142749773692

std_dev = 0.13864750656648953



Output For 4-gram model - 

mean = 1.1077208590691623e-06

std_dev = 9.890162907086684e-06


Output For 5-gram model - 

mean = 2.0472571355320206e-07

std_dev = 1.9243009409389373e-06


Output For 6-gram model - 

mean = 3.889375116558439e-08

std_dev = 3.747037969527212e-07


Output For 7-gram model - 

mean = 7.474282216745883e-09

std_dev = 7.297587965583248e-08


